## Set up the experiment folder

In [1]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'driver-training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('../data/porto_seguro_safe_driver_prediction_input.csv', os.path.join(training_folder, "porto_seguro_safe_driver_prediction_input.csv"))


'driver-training/porto_seguro_safe_driver_prediction_input.csv'

## train.py
This file defines the key functions required to train the model.  
The file can be invoked with `python train.py` for development purposes.

In [2]:
%%writefile $training_folder/train.py
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
import lightgbm


def split_data(data_df):
    """Split a dataframe into training and validation datasets"""

    features = data_df.drop(['target', 'id'], axis=1)
    labels = np.array(data_df['target'])
    features_train, features_valid, labels_train, labels_valid = train_test_split(
        features, labels, test_size=0.2, random_state=0)

    train_data = lightgbm.Dataset(features_train, label=labels_train)
    valid_data = lightgbm.Dataset(
        features_valid, label=labels_valid, free_raw_data=False)

    return (train_data, valid_data)


def train_model(data, parameters):
    """Train a model with the given datasets and parameters"""
    # The object returned by split_data is a tuple.
    # Access train_data with data[0] and valid_data with data[1]

    train_data = data[0]
    valid_data = data[1]
    model = lightgbm.train(parameters,
                           train_data,
                           valid_sets=valid_data,
                           num_boost_round=500,
                           early_stopping_rounds=20)

    return model


def get_model_metrics(model, data):
    """Construct a dictionary of metrics for the model"""

    valid_data = data[1]
    predictions = model.predict(valid_data.data)
    fpr, tpr, thresholds = metrics.roc_curve(valid_data.label, predictions)
    model_metrics = {"auc": (metrics.auc(fpr, tpr))}

    return model_metrics


Overwriting driver-training/train.py


## parameters.json
This file will specify the parameters used to train the model.

In [3]:
%%writefile $training_folder/parameters.json
{
    "training":
    {
        "learning_rate": 0.02,
        "boosting_type": "gbdt",
        "objective": "binary",
        "metric": "auc",
        "sub_feature": 0.7,
        "num_leaves": 60,
        "min_data": 100,
        "min_hessian": 1,
        "verbose": 0
    }
}


Overwriting driver-training/parameters.json


## driver_training.py
This file will be the entry script when running an Azure ML context.  
It calls the functions defined in train.py for data preparation and training, but reads parameters from a file, and logs output to the Azure ML context.  
The file can be invoked with `python driver_training.py` for development purposes.

In [4]:
%%writefile $training_folder/driver_training.py
# Import libraries
import argparse
from azureml.core import Run
import joblib
import json
import os
import pandas as pd
import shutil

# Import functions from train.py
from train import split_data, train_model, get_model_metrics

# Get the output folder for the model from the '--output_folder' parameter
parser = argparse.ArgumentParser()
parser.add_argument('--output_folder', type=str, dest='output_folder', default="outputs")
args = parser.parse_args()
output_folder = args.output_folder

# Get the experiment run context
run = Run.get_context()

# load the safe driver prediction dataset
train_df = pd.read_csv('porto_seguro_safe_driver_prediction_input.csv')

# Load the parameters for training the model from the file
with open("parameters.json") as f:
    pars = json.load(f)
    parameters = pars["training"]

# Log each of the parameters to the run
for param_name, param_value in parameters.items():
    run.parent.log(param_name, param_value) # using `run.parent.log` to log to parent run
    
# Use the functions imported from train.py to prepare data, train the model, and calculate the metrics
data = split_data(train_df)
model = train_model(data, parameters)
model_metrics = get_model_metrics(model, data)
run.parent.log('auc', model_metrics['auc']) # using `run.parent.log` to log to parent run

# Save the trained model to the output folder
os.makedirs(output_folder, exist_ok=True)
output_path = output_folder + "/porto_seguro_safe_driver_model.pkl"
joblib.dump(value=model, filename=output_path)

run.complete()

Overwriting driver-training/driver_training.py
